# 🎾 Плохо настроенный LightGBM

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Функция для имитации реалистичных пропусков в теннисных данных
def create_realistic_tennis_missing_data(tennis_df, random_state=None):
    np.random.seed(random_state)
    df = tennis_df.copy()

    # 1. Рейтинг и очки чаще отсутствуют у малоизвестных игроков
    high_rank_mask = df['Rank_1'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_1'] = np.nan

    high_rank_mask = df['Rank_2'] > 100
    df.loc[high_rank_mask & (np.random.random(len(df)) < 0.3), 'Pts_2'] = np.nan

    # 2. Коэффициенты могут отсутствовать для некоторых матчей
    df.loc[np.random.random(len(df)) < 0.1, ['Odd_1', 'Odd_2']] = np.nan

    # 3. Счет может быть неполным для незавершенных матчей
    # Имитируем отсутствие последних сетов
    def truncate_score(score, prob_missing=0.15):
        if pd.isna(score) or np.random.random() > prob_missing:
            return score

        sets = score.split()
        if len(sets) <= 1:
            return score

        # Сохраняем только первый сет с вероятностью 0.7, иначе первые два
        n_sets = 1 if np.random.random() < 0.7 else 2
        n_sets = min(n_sets, len(sets))

        return ' '.join(sets[:n_sets])

    df['Score'] = df['Score'].apply(lambda x: truncate_score(x))

    return df

# Функция для извлечения признаков из счета матча
def extract_score_features(score):
    if pd.isna(score):
        return [0] * 6
    
    sets = score.split()
    total_sets = len(sets)
    sets_won_1 = 0
    sets_won_2 = 0
    total_games_1 = 0
    total_games_2 = 0
    
    for set_score in sets:
        games = set_score.split('-')
        if len(games) == 2:
            games_1 = int(games[0])
            games_2 = int(games[1])
            total_games_1 += games_1
            total_games_2 += games_2
            if games_1 > games_2:
                sets_won_1 += 1
            else:
                sets_won_2 += 1
    
    return [total_sets, sets_won_1, sets_won_2, 
            total_games_1, total_games_2,
            total_games_1 + total_games_2]

# Функция для создания дополнительных признаков
def create_comparison_features(df):
    features = pd.DataFrame()
    
    # Рейтинговое преимущество (чем больше разница, тем сильнее преимущество)
    features['rank_advantage'] = (df['Rank_2'] - df['Rank_1']) / (df['Rank_1'] + df['Rank_2'])
    
    # Очковое преимущество
    features['points_advantage'] = (df['Pts_1'] - df['Pts_2']) / (df['Pts_1'] + df['Pts_2'])
    
    # Преимущество по коэффициентам
    features['odds_advantage'] = (df['Odd_2'] - df['Odd_1']) / (df['Odd_1'] + df['Odd_2'])
    
    # Бинарные признаки для прямого сравнения
    features['better_rank'] = (df['Rank_1'] < df['Rank_2']).astype(int)
    features['more_points'] = (df['Pts_1'] > df['Pts_2']).astype(int)
    features['better_odds'] = (df['Odd_1'] < df['Odd_2']).astype(int)
    
    # Составной показатель преимущества
    features['overall_advantage'] = (features['better_rank'] + 
                                   features['more_points'] + 
                                   features['better_odds']) / 3
    
    # Добавляем квадраты и взаимодействия признаков
    features['rank_points_interaction'] = features['rank_advantage'] * features['points_advantage']
    features['rank_odds_interaction'] = features['rank_advantage'] * features['odds_advantage']
    features['points_odds_interaction'] = features['points_advantage'] * features['odds_advantage']
    
    return features

# Обновленная функция подготовки данных
def prepare_data(df):
    df = df.copy()
    
    # Конвертируем дату в datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Заменяем отрицательные значения на NaN
    numeric_columns = ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']
    for col in numeric_columns:
        df.loc[df[col] < 0, col] = np.nan
    
    # Извлекаем признаки из счета
    score_features = df['Score'].apply(extract_score_features)
    score_features_df = pd.DataFrame(score_features.tolist(), 
                                   columns=['total_sets', 'sets_won_1', 'sets_won_2',
                                          'total_games_1', 'total_games_2', 'total_games'])
    
    # Создаем признак результата
    df['target'] = (df['Winner'] == df['Player_1']).astype(int)
    
    # Кодируем категориальные признаки
    le = LabelEncoder()
    categorical_columns = ['Series', 'Court', 'Surface', 'Round']
    for col in categorical_columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    # Добавляем новые признаки сравнения
    comparison_features = create_comparison_features(df)
    
    # Объединяем все признаки
    features = pd.concat([
        df[['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']],
        df[[col + '_encoded' for col in categorical_columns]],
        score_features_df,
        comparison_features
    ], axis=1)
    
    return features, df['target']

# Функция для обучения модели с настроенными параметрами
def train_model(df_train):
    X, y = prepare_data(df_train)

    # Заполняем пропущенные значения
    X = X.fillna(X.mean())
    
    # Градиентный бустинг
    model = LGBMClassifier(
        n_estimators=200,           # Количество деревьев
        learning_rate=0.05,         # Скорость обучения
        max_depth=8,                # Ограничение глубины
        num_leaves=31,              # Количество листьев
        min_child_samples=20,       # Минимальное количество образцов в листе
        subsample=0.8,              # Доля выборки для тренировки каждого дерева
        colsample_bytree=0.8,       # Доля признаков для тренировки каждого дерева
        random_state=32,
        class_weight='balanced'     # Балансируем веса классов
    )
    model.fit(X, y)
    
    return model

# Функции get_predictions, create_results_table и display_results остаются без изменений
def get_predictions(model, df_test):
    X_test, _ = prepare_data(df_test)
    X_test = X_test.fillna(X_test.mean())
    probabilities = model.predict_proba(X_test)
    return probabilities

# Функция для создания таблицы результатов
def create_results_table(df_test, predictions):
    results = pd.DataFrame({
        'Player_1': df_test['Player_1'],
        'Player_1_Prediction': predictions[:, 1] * 100,
        'Player_2': df_test['Player_2'],
        'Player_2_Prediction': predictions[:, 0] * 100,
        'Winner': df_test['Winner'],
        'Rank_1': df_test['Rank_1'],
        'Rank_2': df_test['Rank_2'],
        'Pts_1': df_test['Pts_1'],
        'Pts_2': df_test['Pts_2'],
        'Odd_1': df_test['Odd_1'],
        'Odd_2': df_test['Odd_2'],
        'Score': df_test['Score']
    })
    
    # Определяем, кого модель прогнозирует победителем
    results['predicted_winner'] = np.where(predictions[:, 1] > predictions[:, 0], 
                                         df_test['Player_1'], 
                                         df_test['Player_2'])
    
    # Определяем, было ли предсказание верным
    results['correct_prediction'] = (results['predicted_winner'] == df_test['Winner'])
    
    # Добавляем уверенность в прогнозе - максимальная из двух вероятностей
    results['confidence'] = np.maximum(predictions[:, 0], predictions[:, 1]) * 100
    
    return results

# Функция для отображения результатов
def display_results(results):
    styled_results = results.style.format({
        'Player_1_Prediction': '{:.2f}%',
        'Player_2_Prediction': '{:.2f}%',
        'Rank_1': '{:.0f}',
        'Rank_2': '{:.0f}',
        'Pts_1': '{:.0f}',
        'Pts_2': '{:.0f}',
        'Odd_1': '{:.2f}',
        'Odd_2': '{:.2f}'
    }).apply(lambda x: ['background-color: #90EE90' if x['correct_prediction'] 
                       else 'background-color: #FFB6C6' for i in range(len(x))], axis=1)
    
    display(styled_results.hide(axis='columns', subset=['correct_prediction']))

# Функция для оценки и вывода метрик
def display_metrics(model, df_test, predictions, results):
    # Истинные значения (1 - если победил первый игрок, 0 - если второй)
    y_true = (df_test['Winner'] == df_test['Player_1']).astype(int)
    
    # Предсказанные вероятности победы первого игрока
    y_pred_proba = predictions[:, 1]
    
    # Предсказанные бинарные метки
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Расчет метрик
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred) * 100
    recall = recall_score(y_true, y_pred) * 100
    f1 = f1_score(y_true, y_pred) * 100
    roc_auc = roc_auc_score(y_true, y_pred_proba) * 100
    entropy_loss = log_loss(y_true, y_pred_proba)
    
    # Матрица ошибок
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Выводим метрики
    print(f"\nМетрики качества модели:")
    print(f"Accuracy (Точность): {accuracy:.2f}%")
    print(f"Precision (Точность по положительному классу): {precision:.2f}%")
    print(f"Recall (Полнота): {recall:.2f}%")
    print(f"F1-score (F-мера): {f1:.2f}%")
    print(f"ROC AUC: {roc_auc:.2f}%")
    print(f"Log Loss: {entropy_loss:.4f}")
    
    # Анализ ошибок
    print("\nМатрица ошибок (Confusion Matrix):")
    print(f"True Negative: {tn} (Правильно предсказанные победы второго игрока)")
    print(f"False Positive: {fp} (Неправильно предсказанные победы первого игрока)")
    print(f"False Negative: {fn} (Неправильно предсказанные победы второго игрока)")
    print(f"True Positive: {tp} (Правильно предсказанные победы первого игрока)")
    
    # Расчет метрик по уверенности прогноза
    confidence_bins = [50, 60, 70, 80, 90, 100]
    print("\nТочность предсказаний по уровню уверенности:")
    
    for i in range(len(confidence_bins)-1):
        low = confidence_bins[i]
        high = confidence_bins[i+1]
        
        # Фильтруем предсказания по уровню уверенности
        confident_preds = results[(results['confidence'] >= low) & (results['confidence'] < high)]
        
        if len(confident_preds) > 0:
            bin_accuracy = (confident_preds['correct_prediction'].sum() / len(confident_preds)) * 100
            print(f"Уверенность {low}%-{high}%: {bin_accuracy:.2f}% точность ({len(confident_preds)} матчей)")
        else:
            print(f"Уверенность {low}%-{high}%: нет матчей")
    
    # Выводим важность признаков
    feature_names = (
        ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2'] +
        ['Series_encoded', 'Court_encoded', 'Surface_encoded', 'Round_encoded'] +
        ['total_sets', 'sets_won_1', 'sets_won_2', 'total_games_1', 'total_games_2', 'total_games'] +
        ['rank_advantage', 'points_advantage', 'odds_advantage', 'better_rank', 'more_points', 'better_odds', 'overall_advantage', 
         'rank_points_interaction', 'rank_odds_interaction', 'points_odds_interaction']
    )

    # Получаем важность признаков и нормализуем их
    importances = model.feature_importances_
    # Нормализуем важности, чтобы сумма была равна 1
    normalized_importances = importances / importances.sum()
    
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': normalized_importances
    }).sort_values('Importance', ascending=False)
    
    print("\nВажность признаков:")
    print(feature_importance.head(10).to_string(float_format=lambda x: '{:.4f}'.format(x)))

---

## 1. Обучение модели на большой выборке без пропущенных значений

In [2]:
df_train = pd.read_csv('atp_tennis.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

[LightGBM] [Info] Number of positive: 32010, number of negative: 32008
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003969 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3168
[LightGBM] [Info] Number of data points in the train set: 64018, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [3]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.00%,Dolgopolov O.,100.00%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.997742
1,Harrison R.,97.44%,Sela D.,2.56%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,97.442265
2,Anderson K.,97.34%,Edmund K.,2.66%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,97.340877
3,Carreno Busta P.,100.00%,Kubler J.,0.00%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.995998
4,Youzhny M.,0.00%,Cuevas P.,100.00%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.997169
5,Istomin D.,100.00%,Herbert P.H.,0.00%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.997319
6,Delbonis F.,0.00%,Muller G.,100.00%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.997206
7,Seppi A.,100.00%,Moutet C.,0.00%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.997305
8,Ferrer D.,92.90%,Rublev A.,7.10%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,92.898290
9,Ebden M.,100.00%,Isner J.,0.00%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,99.997272



Метрики качества модели:
Accuracy (Точность): 86.18%
Precision (Точность по положительному классу): 78.21%
Recall (Полнота): 100.00%
F1-score (F-мера): 87.77%
ROC AUC: 98.84%
Log Loss: 0.5621

Матрица ошибок (Confusion Matrix):
True Negative: 45 (Правильно предсказанные победы второго игрока)
False Positive: 17 (Неправильно предсказанные победы первого игрока)
False Negative: 0 (Неправильно предсказанные победы второго игрока)
True Positive: 61 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: нет матчей
Уверенность 60%-70%: нет матчей
Уверенность 70%-80%: нет матчей
Уверенность 80%-90%: нет матчей
Уверенность 90%-100%: 86.18% точность (123 матчей)

Важность признаков:
                    Feature  Importance
12               sets_won_2      0.0763
14            total_games_2      0.0730
24    rank_odds_interaction      0.0692
3                     Pts_2      0.0603
11               sets_won_1      0.0602
13            to

### Тестирование на выборке с пропущенными значениями

In [4]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.74%,Dolgopolov O.,99.26%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,99.255347
1,Harrison R.,97.44%,Sela D.,2.56%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,97.442265
2,Anderson K.,97.34%,Edmund K.,2.66%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,97.340877
3,Carreno Busta P.,100.00%,Kubler J.,0.00%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.995998
4,Youzhny M.,0.00%,Cuevas P.,100.00%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.997169
5,Istomin D.,100.00%,Herbert P.H.,0.00%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.997319
6,Delbonis F.,0.00%,Muller G.,100.00%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.997206
7,Seppi A.,8.88%,Moutet C.,91.12%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,91.117076
8,Ferrer D.,92.90%,Rublev A.,7.10%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,92.898290
9,Ebden M.,98.06%,Isner J.,1.94%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,98.056646



Метрики качества модели:
Accuracy (Точность): 86.18%
Precision (Точность по положительному классу): 78.95%
Recall (Полнота): 98.36%
F1-score (F-мера): 87.59%
ROC AUC: 98.04%
Log Loss: 0.5371

Матрица ошибок (Confusion Matrix):
True Negative: 46 (Правильно предсказанные победы второго игрока)
False Positive: 16 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: нет матчей
Уверенность 60%-70%: нет матчей
Уверенность 70%-80%: 0.00% точность (1 матчей)
Уверенность 80%-90%: нет матчей
Уверенность 90%-100%: 86.89% точность (122 матчей)

Важность признаков:
                    Feature  Importance
12               sets_won_2      0.0763
14            total_games_2      0.0730
24    rank_odds_interaction      0.0692
3                     Pts_2      0.0603
11               sets_won_1      0.0602
13

---

## 2. Обучение модели на большой выборке с пропущенными значениями

In [5]:
df_train = pd.read_csv('atp_tennis.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

[LightGBM] [Info] Number of positive: 32010, number of negative: 32008
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3170
[LightGBM] [Info] Number of data points in the train set: 64018, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Размер тренировочного набора: 64018


### Тестирование на выборке без пропущенных значений

In [6]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.01%,Dolgopolov O.,99.99%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.988140
1,Harrison R.,7.61%,Sela D.,92.39%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Sela D.,92.392381
2,Anderson K.,5.51%,Edmund K.,94.49%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,94.488794
3,Carreno Busta P.,99.99%,Kubler J.,0.01%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.989652
4,Youzhny M.,0.02%,Cuevas P.,99.98%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.981597
5,Istomin D.,99.99%,Herbert P.H.,0.01%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.991362
6,Delbonis F.,0.01%,Muller G.,99.99%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.986017
7,Seppi A.,99.99%,Moutet C.,0.01%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,99.991794
8,Ferrer D.,1.46%,Rublev A.,98.54%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,98.541501
9,Ebden M.,99.99%,Isner J.,0.01%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,99.989534



Метрики качества модели:
Accuracy (Точность): 95.12%
Precision (Точность по положительному классу): 100.00%
Recall (Полнота): 90.16%
F1-score (F-мера): 94.83%
ROC AUC: 99.29%
Log Loss: 0.1238

Матрица ошибок (Confusion Matrix):
True Negative: 62 (Правильно предсказанные победы второго игрока)
False Positive: 0 (Неправильно предсказанные победы первого игрока)
False Negative: 6 (Неправильно предсказанные победы второго игрока)
True Positive: 55 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: нет матчей
Уверенность 60%-70%: нет матчей
Уверенность 70%-80%: 0.00% точность (1 матчей)
Уверенность 80%-90%: 75.00% точность (4 матчей)
Уверенность 90%-100%: 96.61% точность (118 матчей)

Важность признаков:
                  Feature  Importance
16         rank_advantage      0.0828
0                  Rank_1      0.0708
6          Series_encoded      0.0597
4                   Odd_1      0.0567
18         odds_advantage      0.056

### Тестирование на выборке с пропущенными значениями

In [7]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,1.05%,Dolgopolov O.,98.95%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,98.947288
1,Harrison R.,7.61%,Sela D.,92.39%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Sela D.,92.392381
2,Anderson K.,5.51%,Edmund K.,94.49%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Edmund K.,94.488794
3,Carreno Busta P.,99.99%,Kubler J.,0.01%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.989652
4,Youzhny M.,0.02%,Cuevas P.,99.98%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,99.981597
5,Istomin D.,99.99%,Herbert P.H.,0.01%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,99.991362
6,Delbonis F.,0.01%,Muller G.,99.99%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,99.986017
7,Seppi A.,33.53%,Moutet C.,66.47%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,66.469805
8,Ferrer D.,1.46%,Rublev A.,98.54%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Rublev A.,98.541501
9,Ebden M.,52.68%,Isner J.,47.32%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Ebden M.,52.677178



Метрики качества модели:
Accuracy (Точность): 94.31%
Precision (Точность по положительному классу): 100.00%
Recall (Полнота): 88.52%
F1-score (F-мера): 93.91%
ROC AUC: 98.84%
Log Loss: 0.1508

Матрица ошибок (Confusion Matrix):
True Negative: 62 (Правильно предсказанные победы второго игрока)
False Positive: 0 (Неправильно предсказанные победы первого игрока)
False Negative: 7 (Неправильно предсказанные победы второго игрока)
True Positive: 54 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 100.00% точность (3 матчей)
Уверенность 60%-70%: 0.00% точность (1 матчей)
Уверенность 70%-80%: 0.00% точность (1 матчей)
Уверенность 80%-90%: 100.00% точность (5 матчей)
Уверенность 90%-100%: 95.58% точность (113 матчей)

Важность признаков:
                  Feature  Importance
16         rank_advantage      0.0828
0                  Rank_1      0.0708
6          Series_encoded      0.0597
4                   Odd_1      0.0567
18 

---

## 3. Обучение модели на маленькой выборке без пропущенных значений

In [8]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Обучаем модель
model = train_model(df_train)
print(f"Размер тренировочного набора: {len(df_train)}")

[LightGBM] [Info] Number of positive: 25, number of negative: 25
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 271
[LightGBM] [Info] Number of data points in the train set: 50, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

### Тестирование на выборке без пропущенных значений

In [9]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,1.03%,Dolgopolov O.,98.97%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,98.971230
1,Harrison R.,77.62%,Sela D.,22.38%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,77.620434
2,Anderson K.,81.32%,Edmund K.,18.68%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,81.324347
3,Carreno Busta P.,99.26%,Kubler J.,0.74%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.257235
4,Youzhny M.,1.61%,Cuevas P.,98.39%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,98.385155
5,Istomin D.,98.46%,Herbert P.H.,1.54%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,98.462932
6,Delbonis F.,2.41%,Muller G.,97.59%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,97.592527
7,Seppi A.,96.71%,Moutet C.,3.29%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,96.713040
8,Ferrer D.,76.98%,Rublev A.,23.02%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,76.975341
9,Ebden M.,76.12%,Isner J.,23.88%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,76.123933



Метрики качества модели:
Accuracy (Точность): 91.87%
Precision (Точность по положительному классу): 86.96%
Recall (Полнота): 98.36%
F1-score (F-мера): 92.31%
ROC AUC: 98.57%
Log Loss: 0.1982

Матрица ошибок (Confusion Matrix):
True Negative: 53 (Правильно предсказанные победы второго игрока)
False Positive: 9 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 100.00% точность (2 матчей)
Уверенность 60%-70%: 42.86% точность (7 матчей)
Уверенность 70%-80%: 75.00% точность (12 матчей)
Уверенность 80%-90%: 87.50% точность (16 матчей)
Уверенность 90%-100%: 98.84% точность (86 матчей)

Важность признаков:
                    Feature  Importance
11               sets_won_1      0.2050
14            total_games_2      0.1500
0                    Rank_1      0.1300
12               sets_won_2     

### Тестирование на выборке с пропущенными значениями

In [10]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,7.77%,Dolgopolov O.,92.23%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,92.232531
1,Harrison R.,77.62%,Sela D.,22.38%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,77.620434
2,Anderson K.,81.32%,Edmund K.,18.68%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,81.324347
3,Carreno Busta P.,99.26%,Kubler J.,0.74%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,99.257235
4,Youzhny M.,1.61%,Cuevas P.,98.39%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,98.385155
5,Istomin D.,98.46%,Herbert P.H.,1.54%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,98.462932
6,Delbonis F.,2.41%,Muller G.,97.59%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,97.592527
7,Seppi A.,43.73%,Moutet C.,56.27%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,56.268585
8,Ferrer D.,76.98%,Rublev A.,23.02%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,76.975341
9,Ebden M.,7.77%,Isner J.,92.23%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,92.232531



Метрики качества модели:
Accuracy (Точность): 88.62%
Precision (Точность по положительному классу): 84.06%
Recall (Полнота): 95.08%
F1-score (F-мера): 89.23%
ROC AUC: 97.03%
Log Loss: 0.2436

Матрица ошибок (Confusion Matrix):
True Negative: 51 (Правильно предсказанные победы второго игрока)
False Positive: 11 (Неправильно предсказанные победы первого игрока)
False Negative: 3 (Неправильно предсказанные победы второго игрока)
True Positive: 58 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 50.00% точность (4 матчей)
Уверенность 60%-70%: 50.00% точность (10 матчей)
Уверенность 70%-80%: 72.73% точность (11 матчей)
Уверенность 80%-90%: 86.67% точность (15 матчей)
Уверенность 90%-100%: 97.59% точность (83 матчей)

Важность признаков:
                    Feature  Importance
11               sets_won_1      0.2050
14            total_games_2      0.1500
0                    Rank_1      0.1300
12               sets_won_2    

---

## 4. Обучение модели на маленькой выборке с пропущенными значениями

In [11]:
df_train = pd.read_csv('atp_tennis_47302_xxs.csv')

# Имитируем пропущенные значения в обучающей выборке
df_train_missing = create_realistic_tennis_missing_data(df_train, random_state=32)

# Обучаем модель
model = train_model(df_train_missing)
print(f"Размер тренировочного набора: {len(df_train)}")

[LightGBM] [Info] Number of positive: 25, number of negative: 25
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 257
[LightGBM] [Info] Number of data points in the train set: 50, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

### Тестирование на выборке без пропущенных значений

In [12]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Получаем предсказания
predictions = get_predictions(model, df_test)
    
# Создаем таблицу результатов
results = create_results_table(df_test, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,0.37%,Dolgopolov O.,99.63%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7 3-6,Dolgopolov O.,99.627928
1,Harrison R.,90.03%,Sela D.,9.97%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,90.029285
2,Anderson K.,95.87%,Edmund K.,4.13%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,95.867037
3,Carreno Busta P.,97.76%,Kubler J.,2.24%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,97.760288
4,Youzhny M.,1.11%,Cuevas P.,98.89%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,98.892849
5,Istomin D.,97.35%,Herbert P.H.,2.65%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,97.353273
6,Delbonis F.,1.30%,Muller G.,98.70%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,98.702963
7,Seppi A.,98.07%,Moutet C.,1.93%,Seppi A.,76,155,686,361,1.33,3.40,3-6 6-4 6-2,Seppi A.,98.074517
8,Ferrer D.,88.00%,Rublev A.,12.00%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,87.995041
9,Ebden M.,70.38%,Isner J.,29.62%,Ebden M.,78,16,684,2265,3.50,1.30,6-4 3-6 6-3,Ebden M.,70.378814



Метрики качества модели:
Accuracy (Точность): 86.99%
Precision (Точность по положительному классу): 80.00%
Recall (Полнота): 98.36%
F1-score (F-мера): 88.24%
ROC AUC: 97.98%
Log Loss: 0.2491

Матрица ошибок (Confusion Matrix):
True Negative: 47 (Правильно предсказанные победы второго игрока)
False Positive: 15 (Неправильно предсказанные победы первого игрока)
False Negative: 1 (Неправильно предсказанные победы второго игрока)
True Positive: 60 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 0.00% точность (5 матчей)
Уверенность 60%-70%: 33.33% точность (3 матчей)
Уверенность 70%-80%: 60.00% точность (5 матчей)
Уверенность 80%-90%: 71.43% точность (7 матчей)
Уверенность 90%-100%: 95.15% точность (103 матчей)

Важность признаков:
                    Feature  Importance
11               sets_won_1      0.3850
23  rank_points_interaction      0.1250
4                     Odd_1      0.1150
0                    Rank_1      0

### Тестирование на выборке с пропущенными значениями

In [13]:
df_test = pd.read_csv('atp_tennis_test_minus_1_set_47967_big.csv')

# Имитируем пропущенные значения в тестовой выборке
df_test_missing = create_realistic_tennis_missing_data(df_test, random_state=32)

# Получаем предсказания
predictions = get_predictions(model, df_test_missing)
    
# Создаем таблицу результатов
results = create_results_table(df_test_missing, predictions)
    
# Отображаем результаты
display_results(results)
    
# Отображаем подробные метрики качества модели
display_metrics(model, df_test_missing, predictions, results)

,Player_1,Player_1_Prediction,Player_2,Player_2_Prediction,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score,predicted_winner,confidence
0,Haider-Maurer A.,1.94%,Dolgopolov O.,98.06%,Dolgopolov O.,448,37,81,1296,13.00,1.04,6-7,Dolgopolov O.,98.063370
1,Harrison R.,90.03%,Sela D.,9.97%,Harrison R.,45,95,1115,585,1.30,3.50,6-3 5-7 3-6 7-5,Harrison R.,90.029285
2,Anderson K.,95.87%,Edmund K.,4.13%,Edmund K.,12,49,2610,992,1.40,3.00,7-6 3-6 6-3 3-6,Anderson K.,95.867037
3,Carreno Busta P.,97.76%,Kubler J.,2.24%,Carreno Busta P.,11,243,2615,217,1.28,3.75,7-5 4-6 7-5,Carreno Busta P.,97.760288
4,Youzhny M.,1.11%,Cuevas P.,98.89%,Cuevas P.,90,34,604,1345,2.10,1.72,6-7 3-6,Cuevas P.,98.892849
5,Istomin D.,97.35%,Herbert P.H.,2.65%,Istomin D.,60,74,848,690,1.66,2.20,6-2 6-1 5-7,Istomin D.,97.353273
6,Delbonis F.,1.30%,Muller G.,98.70%,Muller G.,67,28,755,1490,4.00,1.25,5-7 4-6,Muller G.,98.702963
7,Seppi A.,33.20%,Moutet C.,66.80%,Seppi A.,76,155,686,361,1.33,3.40,3-6,Moutet C.,66.803317
8,Ferrer D.,88.00%,Rublev A.,12.00%,Rublev A.,33,32,1360,1373,1.90,1.90,5-7 7-6 2-6 7-6,Ferrer D.,87.995041
9,Ebden M.,2.27%,Isner J.,97.73%,Ebden M.,78,16,684,2265,3.50,1.30,6-4,Isner J.,97.734482



Метрики качества модели:
Accuracy (Точность): 85.37%
Precision (Точность по положительному классу): 80.28%
Recall (Полнота): 93.44%
F1-score (F-мера): 86.36%
ROC AUC: 95.58%
Log Loss: 0.3047

Матрица ошибок (Confusion Matrix):
True Negative: 48 (Правильно предсказанные победы второго игрока)
False Positive: 14 (Неправильно предсказанные победы первого игрока)
False Negative: 4 (Неправильно предсказанные победы второго игрока)
True Positive: 57 (Правильно предсказанные победы первого игрока)

Точность предсказаний по уровню уверенности:
Уверенность 50%-60%: 16.67% точность (6 матчей)
Уверенность 60%-70%: 25.00% точность (4 матчей)
Уверенность 70%-80%: 60.00% точность (5 матчей)
Уверенность 80%-90%: 75.00% точность (8 матчей)
Уверенность 90%-100%: 94.00% точность (100 матчей)

Важность признаков:
                    Feature  Importance
11               sets_won_1      0.3850
23  rank_points_interaction      0.1250
4                     Odd_1      0.1150
0                    Rank_1      